In [295]:
import openai
import numpy as np
import os
from dotenv import load_dotenv
import os
from collections import deque
from typing import Dict, List, Optional, Any
from geopy.geocoders import Nominatim
from typing import Any, Callable, Dict, List, Optional, Sequence, Tuple, Union

from pydantic import BaseModel, Field
from datetime import datetime
from typing import Optional

from langchain import LLMChain, OpenAI, PromptTemplate
from langchain.embeddings import OpenAIEmbeddings
from langchain.llms import BaseLLM
from langchain.vectorstores.base import VectorStore
from pydantic import BaseModel, Field
from langchain.chains.base import Chain
# Langchain
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Pinecone
from langchain.document_loaders import TextLoader
from langchain.chat_models import ChatOpenAI
from langchain.chains import RetrievalQA, LLMChain ,LLMCheckerChain
from langchain.callbacks import wandb_tracing_enabled
from langchain.prompts import (
    PromptTemplate,
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
    StringPromptTemplate
)
from langchain.prompts.few_shot import FewShotPromptTemplate
from langchain.output_parsers import StructuredOutputParser, ResponseSchema

from typing import Optional
from langchain.chains import SimpleSequentialChain ,SequentialChain
from langchain.agents import AgentExecutor, Tool, ZeroShotAgent,BaseMultiActionAgent
from langchain.agents import AgentType, initialize_agent,AgentExecutor,BaseSingleActionAgent
from langchain.tools import tool
from langchain.chains.openai_functions import (
    create_openai_fn_chain,
    create_structured_output_chain,
)
from langchain.schema import HumanMessage, AIMessage, ChatMessage
from langchain.vectorstores import FAISS
from langchain.embeddings import OpenAIEmbeddings
from langchain.schema import Document
from langchain.agents import Tool, AgentExecutor, LLMSingleActionAgent, AgentOutputParser,Agent
from langchain.prompts import StringPromptTemplate
from langchain import OpenAI, SerpAPIWrapper, LLMChain
from typing import List, Union
from langchain.schema import AgentAction, AgentFinish, OutputParserException
import re
from langchain.agents import Tool, AgentExecutor, BaseSingleActionAgent
from langchain import OpenAI, SerpAPIWrapper

from langchain.callbacks.manager import (
    AsyncCallbackManagerForChainRun,
    AsyncCallbackManagerForToolRun,
    CallbackManagerForChainRun,
    CallbackManagerForToolRun,
    Callbacks,
)
from utils.Webscraper import Webscraper
webscraper = Webscraper()

In [296]:
# Load variables from the .env file
load_dotenv('./.env')

# Access the variables
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")
INDEX_NAME = os.getenv("PINECONE_INDEX_NAME")
LANGCHAIN_PROJECT = os.getenv("LANGCHAIN_PROJECT")
LANGCHAIN_API_KEY = os.getenv("LANGCHAIN_API_KEY")
LANGCHAIN_TRACING_V2=os.getenv("LANGCHAIN_TRACING_V2")
LANGCHAIN_ENDPOINT=os.getenv("LANGCHAIN_ENDPOINT")

# Set the environment variables
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY
os.environ["LANGCHAIN_TRACING_V2"] = LANGCHAIN_TRACING_V2
os.environ["LANGCHAIN_ENDPOINT"] = LANGCHAIN_ENDPOINT
os.environ["LANGCHAIN_API_KEY"] = LANGCHAIN_API_KEY
os.environ["LANGCHAIN_PROJECT"] = LANGCHAIN_PROJECT

## Setup tools for location extraction 

In [297]:
@tool("get_location",return_direct=False)
def get_coordinates(location:str) -> tuple[float, float]:
    """Returns the latitude and longitude of a location.Location should include any landmarks,cities, countries and addresses, output an address searchable in googleMaps be as specific as possible."""
    geolocator = Nominatim(user_agent="geoapi_explorer")
    try:
        location_info = geolocator.geocode(location)
    except:
        return "Location Input must be more general, E.g Country,City,state,street"
    
    if location_info:
        latitude = location_info.latitude
        longitude = location_info.longitude
        return (latitude, longitude)
    else:
        return "Location Input must be more general, E.g Country,City,state,street"

@tool("get_article",return_direct=False)
def get_article(n)->str:
    """Returns the article text of the article we want to analyze to extract location, Disruption type and Severity."""
    article2 = webscraper.scrapeSingleUrl("https://www.straitstimes.com/asia/australianz/56-magnitude-quake-hits-central-new-zealand")
    article_text_title = article2.title+article2.text

    return article_text_title

tools = [
    get_coordinates,
    get_article,
]
tool_names = [tool.name for tool in tools]
# use the function
coordinates = get_coordinates("Bali, Indonesia")
print(f'coordinates',coordinates)

article_text_title = get_article('dqKWFIWVWJERINBER')
print(f'article_text_title',article_text_title)

coordinates (-8.3304977, 115.0906401)
article_text_title 5.6-magnitude quake hits central New ZealandWELLINGTON - An earthquake with a magnitude of 5.6 jolted 75km north of French Pass, central New Zealand, at 11.53am local time (7.53am Singapore time) on Friday, according to the country’s geological hazard monitoring system GeoNet.  Residents in Wellington and other central New Zealand regions felt the shake, which was depicted as a “light earthquake” with a depth of 170km.  The quake struck as thousands of football fans were heading to Wellington Regional Stadium for the Women’s World Cup quarter-final clash between Spain and the Netherlands, which kicks off at 1pm, the New Zealand (NZ) Herald reported.  The tremor was felt widely across parts of the North Island and the upper South Island, with more than 20,000 reports from people, according to GeoNet.  In a tweet, GeoNet said, “shaking like this is a good reminder that earthquakes can occur anywhere in New Zealand at any time”.  “I

## Define the custom agents + LLMs 

In [298]:
class Severity(BaseModel):
    """Base class to represent the severity of a disruption event.Includes common properties of severity."""
    Duration: int = Field(..., description="Estimated duration of imapct casued by the disruption event on the location in Days.")
    Casualties: Optional[int] = Field(..., description="Estimated number of casualties caused by the disruption event.")
    Deaths: Optional[int] = Field(..., description="Estimated number of deaths caused by the disruption event.")
    DamageCost: Optional[float] = Field(..., description="Estimated cost of damage caused by the disruption event.")

sample_final_output = {
    "Location": {
        "Name": "Fukushima Daiichi nuclear plant",
        "Coordinates": (37.4214, 141.0324),
    },
    "DisruptionType": "Earthquake",
    "Severity": {
        "Deaths": 0, # optional
        "Duration": 3, # optional
        "Casualties": 0, # Required
        "DamageCost": 0, # optional
    }
}

In [299]:
schema ={
  "name": "AgentOutputSchema",
  "description":"The output of the agent,must be used",
  "type": "object",
  "properties": {
    "Thought": {
      "type": "string",
      "description": "A thought or consideration."
    },
    "Action": {
      "type": "string",
      "enum": ["tool_names"],
      "description": "The action to take, should be one of the specified tool names."
    },
    "ActionInput": {
      "type": "string",
      "description": "The input to the action."
    },
    "Observation": {
      "type": "string",
      "description": "The result of the action."
    },
    "FinalAnswer": {
      "type": ["object", "null"],
      "description": "The final answer in the format (latitude, longitude).",
      "properties": {
        "Location": {
          "type": "object",
          "properties": {
            "Name": {
              "type": "string",
              "description": "Name of the location."
            },
            "Coordinates": {
              "type": "object",
              "properties": {
                "Latitude": {
                  "type": "number",
                  "minimum": -90,
                  "maximum": 90,
                  "description": "Latitude"
                },
                "Longitude": {
                  "type": "number",
                  "minimum": -180,
                  "maximum": 180,
                  "description": "Longitude"
                }
              },
              "required": ["Latitude", "Longitude"],
              "description": "Coordinates in the format (latitude, longitude)."
            }
          },
          "required": ["Name", "Coordinates"],
          "description": "Location information."
        },
        "DisruptionType": {
          "type": "string",
          "description": "Type of disruption event."
        },
        "Severity": {
          "type": "string",
          "description": "Quantifiable Severity metrics of the disruption event."
        }
      },
      "required": ["Location", "DisruptionType", "Severity"]
    }
  },
  "required": ["Thought", "Action", "ActionInput", "Observation"]
}

In [270]:
"""You're a geographical expert aimed to identity the location(s) about a singular disruption event from News article."""

"You're a geographical expert aimed to identity the location(s) about a singular disruption event from News article."

In [271]:
template = """Role: You are a Disruption event News Analyst, your goal is to extract data from news articles such as Location and Disruption Type and Quantifiable Severity Metrics. You have access to the following tools:

{tools}

Must Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: 1.coordinates of the location in the format (latitude, longitude) and 2. The type of disruption event 3. Quantifiable Severity Metrics 

Task: {task}
{agent_scratchpad}"""

# Set up a prompt template
class CustomPromptTemplate(StringPromptTemplate):
    # The template to use
    template: str
    # The list of tools available
    tools: List[Tool]

    def format(self, **kwargs) -> str:
        # Get the intermediate steps (AgentAction, Observation tuples)
        # Format them in a particular way
        intermediate_steps = kwargs.pop("intermediate_steps")
        thoughts = ""
        for action, observation in intermediate_steps:
            print(f'action: {action},action.log: {action.log},observation: {observation}')
            thoughts += action.log['Thought']
            thoughts += f"\nObservation: {observation}\nThought: "
        # Set the agent_scratchpad variable to that value
        kwargs["agent_scratchpad"] = thoughts
        # Create a tools variable from the list of tools provided
        kwargs["tools"] = "\n".join([f"{tool.name}: {tool.description}" for tool in self.tools])
        # Create a list of tool names for the tools provided
        kwargs["tool_names"] = ", ".join([tool.name for tool in self.tools])
        return self.template.format(**kwargs)
    
prompt = CustomPromptTemplate(
    template=template,
    tools=tools,
    # This omits the `agent_scratchpad`, `tools`, and `tool_names` variables because those are generated dynamically
    # This includes the `intermediate_steps` variable because that is needed
    input_variables=["intermediate_steps",'task']
)
model_name = "gpt-4-0613"
temperature = 1
llm = ChatOpenAI(model_name=model_name, temperature=temperature)
llm.to_json()

# LLM chain consisting of the LLM and a prompt
llm_chain = create_structured_output_chain(llm=llm, prompt=prompt,output_schema=schema)



In [272]:
class CustomOutputParser(AgentOutputParser):
    def parse(self, llm_output: dict) -> Union[AgentAction, AgentFinish]:
        print(f'llm_output: {llm_output}, type: {type(llm_output)}, keys: {llm_output.keys()}')
        # Check if agent should finish, 
        if "FinalAnswer" in llm_output.keys():
            print(f'FinalAnswer: {llm_output["FinalAnswer"]},keys: {llm_output.keys()}')
            return AgentFinish(
                # Return values is generally always a dictionary with a single `output` key
                # It is not recommended to try anything else at the moment :)
                return_values={"output": llm_output["FinalAnswer"]},
                log=llm_output,
            )
        # Parse out the action and action input
        action = llm_output["Action"]
        action_input = llm_output["ActionInput"]
        print(f'action: {action}, action_input: {action_input}')

        return AgentAction(tool=action, tool_input=action_input.strip(" ").strip('"'), log=llm_output)
    
output_parser = CustomOutputParser()

In [273]:
class LLMSingleActionAgent(BaseSingleActionAgent):
    """Base class for single action agents."""
    
    llm_chain: LLMChain
    """LLMChain to use for agent."""
    output_parser: AgentOutputParser
    """Output parser to use for agent."""
    stop: List[str]
    """List of strings to stop on."""

    @property
    def input_keys(self) -> List[str]:
        """Return the input keys.

        Returns:
            List of input keys.
        """
        return list(set(self.llm_chain.input_keys) - {"intermediate_steps"})

    def dict(self, **kwargs: Any) -> Dict:
        """Return dictionary representation of agent."""
        _dict = super().dict()
        del _dict["output_parser"]
        return _dict

    def plan(
        self,
        intermediate_steps: List[Tuple[AgentAction, str]],
        callbacks: Callbacks = None,
        **kwargs: Any,
    ) -> Union[AgentAction, AgentFinish]:
        """Given input, decided what to do.

        Args:
            intermediate_steps: Steps the LLM has taken to date,
                along with the observations.
            callbacks: Callbacks to run.
            **kwargs: User inputs.

        Returns:
            Action specifying what tool to use.
        """
        output = self.llm_chain.run(
            intermediate_steps=intermediate_steps,
            stop=self.stop,
            callbacks=callbacks,
            **kwargs,
        )
        return self.output_parser.parse(output)
    
    async def aplan(
        self,
        intermediate_steps: List[Tuple[AgentAction, str]],
        callbacks: Callbacks = None,
        **kwargs: Any,
    ) -> Union[AgentAction, AgentFinish]:
        """Given input, decided what to do.
        
        Args:
            intermediate_steps: Steps the LLM has taken to date,
                along with observations
            callbacks: Callbacks to run.
            **kwargs: User inputs.

        Returns:
            Action specifying what tool to use.
        """
        output = await self.llm_chain.arun(
            intermediate_steps=intermediate_steps,
            stop=self.stop,
            callbacks=callbacks,
            **kwargs,
        )
        return self.output_parser.parse(output)

    def tool_run_logging_kwargs(self) -> Dict:
        return {
            "llm_prefix": "",
            "observation_prefix": "" if len(self.stop) == 0 else self.stop[0],
        }
    
agent = LLMSingleActionAgent(
    llm_chain=llm_chain,
    output_parser=output_parser,
    stop=["\nObservation:"],
    allowed_tools=tool_names
)

Filtering News article chain

In [292]:
llm = ChatOpenAI(model_name="gpt-3.5-turbo-0613", temperature=1)
classifier_article_schema = {
    "name": "binary_classifier_article_schema",
    "description": "Binary Classifier Schema for Article, 0 for False and 1 for True",
    "type": "object",
    "properties": {
      "isDisruptionEvent": {
        "type": "boolean"
      },
      "Reason": {
        "type": "string"
      }
    },
    "required": ["isDisruptionEvent", "Reason"]
  }

classifierprompt = PromptTemplate(
    template = """Role:You are a Binary Classifier,your goal is to classify if the given news article is a vaild disruption event article or not.
    Conditions:
    1. A disruption event can be a Natural Disaster Example but not limited to: Earthquake,Flood,Extreme Weather. OR Man-Made but not limited to: Geo-Political,Protest/Riot,Airport Disruption.
    2. The disruption event the news article is reporting, must be a 'live' event, meaning it is currently happening. Not an article reporting on a past event.

    Article: {article}

    TASK: Given youre Role and the Conditions, Classify if the given news article is a vaild disruption event article or not. Think through and give reasoning for your decision. Must Output 0 for False and 1 for True.
    """,
    input_variables=["article"]
)

ArticleClassifier = create_structured_output_chain(output_schema=classifier_article_schema,llm = llm,prompt=classifierprompt)

In [293]:
ArticleClassifier.run(article=article_text_title)

{'isDisruptionEvent': True,
 'Reason': 'The news article reports a 5.6-magnitude earthquake in central New Zealand, which is a natural disaster and satisfies the condition of a disruption event. Additionally, the article mentions that the earthquake was felt by residents and coincided with a football match, indicating that it is a live event. Therefore, the given news article is a valid disruption event article.'}

## Agent Excutor Code

In [274]:
from __future__ import annotations

import asyncio
import json
import logging
import time
from abc import abstractmethod
from pathlib import Path
from typing import Any, Callable, Dict, List, Optional, Sequence, Tuple, Union

import yaml

from langchain.agents.agent_iterator import AgentExecutorIterator
from langchain.agents.agent_types import AgentType
from langchain.agents.tools import InvalidTool
from langchain.callbacks.base import BaseCallbackManager
from langchain.callbacks.manager import (
    AsyncCallbackManagerForChainRun,
    AsyncCallbackManagerForToolRun,
    CallbackManagerForChainRun,
    CallbackManagerForToolRun,
    Callbacks,
)
from langchain.chains.base import Chain
from langchain.chains.llm import LLMChain
from langchain.prompts.few_shot import FewShotPromptTemplate
from langchain.prompts.prompt import PromptTemplate
from langchain.pydantic_v1 import BaseModel, root_validator
from langchain.schema import (
    AgentAction,
    AgentFinish,
    BaseOutputParser,
    BasePromptTemplate,
    OutputParserException,
)
from langchain.schema.language_model import BaseLanguageModel
from langchain.schema.messages import BaseMessage
from langchain.tools.base import BaseTool
from langchain.utilities.asyncio import asyncio_timeout
from langchain.utils.input import get_color_mapping
from langchain.agents.agent import  ExceptionTool
logger = logging.getLogger(__name__)

class Instruction(BaseModel):
    """Defines the instruction and the llmchain to run"""
    task: str = Field(..., description="The name of the task to perform.")
    llmChain: LLMChain = Field(..., description="The LLMChain to use for the task.")

class AgentExecutor2(Chain):
    """Agent that is using tools."""

    instruction_manuel: List[Instruction] = Field(..., description="The list of instructions to perform.")

    agent: Union[BaseSingleActionAgent, BaseMultiActionAgent]
    """The agent to run for creating a plan and determining actions
    to take at each step of the execution loop."""
    tools: Sequence[BaseTool]
    """The valid tools the agent can call."""
    return_intermediate_steps: bool = False
    """Whether to return the agent's trajectory of intermediate steps
    at the end in addition to the final output."""
    max_iterations: Optional[int] = 15
    """The maximum number of steps to take before ending the execution
    loop.
    
    Setting to 'None' could lead to an infinite loop."""
    max_execution_time: Optional[float] = None
    """The maximum amount of wall clock time to spend in the execution
    loop.
    """
    early_stopping_method: str = "force"
    """The method to use for early stopping if the agent never
    returns `AgentFinish`. Either 'force' or 'generate'.

    `"force"` returns a string saying that it stopped because it met a
        time or iteration limit.
    
    `"generate"` calls the agent's LLM Chain one final time to generate
        a final answer based on the previous steps.
    """
    handle_parsing_errors: Union[
        bool, str, Callable[[OutputParserException], str]
    ] = False
    """How to handle errors raised by the agent's output parser.
    Defaults to `False`, which raises the error.
s
    If `true`, the error will be sent back to the LLM as an observation.
    If a string, the string itself will be sent to the LLM as an observation.
    If a callable function, the function will be called with the exception
     as an argument, and the result of that function will be passed to the agent
      as an observation.
    """
    trim_intermediate_steps: Union[
        int, Callable[[List[Tuple[AgentAction, str]]], List[Tuple[AgentAction, str]]]
    ] = -1

    @classmethod
    def from_agent_and_tools(
        cls,
        agent: Union[BaseSingleActionAgent, BaseMultiActionAgent],
        tools: Sequence[BaseTool],
        callback_manager: Optional[BaseCallbackManager] = None,
        **kwargs: Any,
    ) -> AgentExecutor:
        """Create from agent and tools."""
        return cls(
            agent=agent, tools=tools, callback_manager=callback_manager, **kwargs
        )

    @root_validator()
    def validate_tools(cls, values: Dict) -> Dict:
        """Validate that tools are compatible with agent."""
        agent = values["agent"]
        tools = values["tools"]
        allowed_tools = agent.get_allowed_tools()
        if allowed_tools is not None:
            if set(allowed_tools) != set([tool.name for tool in tools]):
                raise ValueError(
                    f"Allowed tools ({allowed_tools}) different than "
                    f"provided tools ({[tool.name for tool in tools]})"
                )
        return values

    @root_validator()
    def validate_return_direct_tool(cls, values: Dict) -> Dict:
        """Validate that tools are compatible with agent."""
        agent = values["agent"]
        tools = values["tools"]
        if isinstance(agent, BaseMultiActionAgent):
            for tool in tools:
                if tool.return_direct:
                    raise ValueError(
                        "Tools that have `return_direct=True` are not allowed "
                        "in multi-action agents"
                    )
        return values

    def save(self, file_path: Union[Path, str]) -> None:
        """Raise error - saving not supported for Agent Executors."""
        raise ValueError(
            "Saving not supported for agent executors. "
            "If you are trying to save the agent, please use the "
            "`.save_agent(...)`"
        )

    def save_agent(self, file_path: Union[Path, str]) -> None:
        """Save the underlying agent."""
        return self.agent.save(file_path)

    def iter(
        self,
        inputs: Any,
        callbacks: Callbacks = None,
        *,
        include_run_info: bool = False,
        async_: bool = False,
    ) -> AgentExecutorIterator:
        """Enables iteration over steps taken to reach final output."""
        return AgentExecutorIterator(
            self,
            inputs,
            callbacks,
            tags=self.tags,
            include_run_info=include_run_info,
            async_=async_,
        )

    @property
    def input_keys(self) -> List[str]:
        """Return the input keys.

        :meta private:
        """
        return self.agent.input_keys

    @property
    def output_keys(self) -> List[str]:
        """Return the singular output key.

        :meta private:
        """
        if self.return_intermediate_steps:
            return self.agent.return_values + ["intermediate_steps"]
        else:
            return self.agent.return_values

    def lookup_tool(self, name: str) -> BaseTool:
        """Lookup tool by name."""
        return {tool.name: tool for tool in self.tools}[name]

    def _should_continue(self, iterations: int, time_elapsed: float,current_instruction_index) -> bool:
        if self.max_iterations is not None and iterations >= self.max_iterations:
            return False
        if (
            self.max_execution_time is not None
            and time_elapsed >= self.max_execution_time
        ):
            return False
        
        # Continue if there are still instructions to execute
        if current_instruction_index == len(self.instruction_manuel):
            return False
            
        

        return True

    def _return(
        self,
        output: AgentFinish,
        intermediate_steps: list,
        run_manager: Optional[CallbackManagerForChainRun] = None,
    ) -> Dict[str, Any]:
        print(f'_return: {output}')
        if run_manager:
            run_manager.on_agent_finish(output, color="green", verbose=self.verbose)
        final_output = output.return_values
        if self.return_intermediate_steps:
            final_output["intermediate_steps"] = intermediate_steps
        return final_output

    async def _areturn(
        self,
        output: AgentFinish,
        intermediate_steps: list,
        run_manager: Optional[AsyncCallbackManagerForChainRun] = None,
    ) -> Dict[str, Any]:
        print(f'async def _return output: {output}')
        if run_manager:
            await run_manager.on_agent_finish(
                output, color="green", verbose=self.verbose
            )
        final_output = output.return_values
        if self.return_intermediate_steps:
            final_output["intermediate_steps"] = intermediate_steps
        return final_output

    def _take_next_step(
        self,
        name_to_tool_map: Dict[str, BaseTool],
        color_mapping: Dict[str, str],
        inputs: Dict[str, str],
        intermediate_steps: List[Tuple[AgentAction, str]],
        run_manager: Optional[CallbackManagerForChainRun] = None,
    ) -> Union[AgentFinish, List[Tuple[AgentAction, str]]]:
        """Take a single step in the thought-action-observation loop.

        Override this to take control of how the agent makes and acts on choices.
        """
        print(f'def _take_next_step inputs: {inputs}')
        try:
            intermediate_steps = self._prepare_intermediate_steps(intermediate_steps)
            print('intermediate_steps: ', intermediate_steps)

            # Call the LLM to see what to do.
            output = self.agent.plan(
                intermediate_steps,
                callbacks=run_manager.get_child() if run_manager else None,
                **inputs,
            )
        except OutputParserException as e:
            if isinstance(self.handle_parsing_errors, bool):
                raise_error = not self.handle_parsing_errors
            else:
                raise_error = False
            if raise_error:
                raise e
            text = str(e)
            if isinstance(self.handle_parsing_errors, bool):
                if e.send_to_llm:
                    observation = str(e.observation)
                    text = str(e.llm_output)
                else:
                    observation = "Invalid or incomplete response"
            elif isinstance(self.handle_parsing_errors, str):
                observation = self.handle_parsing_errors
            elif callable(self.handle_parsing_errors):
                observation = self.handle_parsing_errors(e)
            else:
                raise ValueError("Got unexpected type of `handle_parsing_errors`")
            output = AgentAction("_Exception", observation, text)
            if run_manager:
                run_manager.on_agent_action(output, color="green")
            tool_run_kwargs = self.agent.tool_run_logging_kwargs()
            observation = ExceptionTool().run(
                output.tool_input,
                verbose=self.verbose,
                color=None,
                callbacks=run_manager.get_child() if run_manager else None,
                **tool_run_kwargs,
            )
            return [(output, observation)]
        # If the tool chosen is the finishing tool, then we end and return.
        if isinstance(output, AgentFinish):
            return output
        actions: List[AgentAction]
        if isinstance(output, AgentAction):
            actions = [output]
        else:
            actions = output
        result = []
        print('result: ', result)
        for agent_action in actions:
            print('agent_action: ', agent_action)
            if run_manager:
                run_manager.on_agent_action(agent_action, color="green")
            # Otherwise we lookup the tool
            if agent_action.tool in name_to_tool_map:
                tool = name_to_tool_map[agent_action.tool]
                return_direct = tool.return_direct
                color = color_mapping[agent_action.tool]
                tool_run_kwargs = self.agent.tool_run_logging_kwargs()
                if return_direct:
                    tool_run_kwargs["llm_prefix"] = ""
                # We then call the tool on the tool input to get an observation
                observation = tool.run(
                    agent_action.tool_input,
                    verbose=self.verbose,
                    color=color,
                    callbacks=run_manager.get_child() if run_manager else None,
                    **tool_run_kwargs,
                )
            else:
                tool_run_kwargs = self.agent.tool_run_logging_kwargs()
                observation = InvalidTool().run(
                    {
                        "requested_tool_name": agent_action.tool,
                        "available_tool_names": list(name_to_tool_map.keys()),
                    },
                    verbose=self.verbose,
                    color=None,
                    callbacks=run_manager.get_child() if run_manager else None,
                    **tool_run_kwargs,
                )
            result.append((agent_action, observation))
        return result

    async def _atake_next_step(
        self,
        name_to_tool_map: Dict[str, BaseTool],
        color_mapping: Dict[str, str],
        inputs: Dict[str, str],
        intermediate_steps: List[Tuple[AgentAction, str]],
        run_manager: Optional[AsyncCallbackManagerForChainRun] = None,
    ) -> Union[AgentFinish, List[Tuple[AgentAction, str]]]:
        """Take a single step in the thought-action-observation loop.

        Override this to take control of how the agent makes and acts on choices.
        """
        print(f'async def _take_next_step inputs: {inputs}')
        try:
            intermediate_steps = self._prepare_intermediate_steps(intermediate_steps)

            # Call the LLM to see what to do.
            output = await self.agent.aplan(
                intermediate_steps,
                callbacks=run_manager.get_child() if run_manager else None,
                **inputs,
            )
        except OutputParserException as e:
            if isinstance(self.handle_parsing_errors, bool):
                raise_error = not self.handle_parsing_errors
            else:
                raise_error = False
            if raise_error:
                raise e
            text = str(e)
            if isinstance(self.handle_parsing_errors, bool):
                if e.send_to_llm:
                    observation = str(e.observation)
                    text = str(e.llm_output)
                else:
                    observation = "Invalid or incomplete response"
            elif isinstance(self.handle_parsing_errors, str):
                observation = self.handle_parsing_errors
            elif callable(self.handle_parsing_errors):
                observation = self.handle_parsing_errors(e)
            else:
                raise ValueError("Got unexpected type of `handle_parsing_errors`")
            output = AgentAction("_Exception", observation, text)
            tool_run_kwargs = self.agent.tool_run_logging_kwargs()
            observation = await ExceptionTool().arun(
                output.tool_input,
                verbose=self.verbose,
                color=None,
                callbacks=run_manager.get_child() if run_manager else None,
                **tool_run_kwargs,
            )
            return [(output, observation)]
        # If the tool chosen is the finishing tool, then we end and return.
        if isinstance(output, AgentFinish):
            return output
        actions: List[AgentAction]
        if isinstance(output, AgentAction):
            actions = [output]
        else:
            actions = output

        async def _aperform_agent_action(
            agent_action: AgentAction,
        ) -> Tuple[AgentAction, str]:
            if run_manager:
                await run_manager.on_agent_action(
                    agent_action, verbose=self.verbose, color="green"
                )
            # Otherwise we lookup the tool
            if agent_action.tool in name_to_tool_map:
                tool = name_to_tool_map[agent_action.tool]
                return_direct = tool.return_direct
                color = color_mapping[agent_action.tool]
                tool_run_kwargs = self.agent.tool_run_logging_kwargs()
                if return_direct:
                    tool_run_kwargs["llm_prefix"] = ""
                # We then call the tool on the tool input to get an observation
                observation = await tool.arun(
                    agent_action.tool_input,
                    verbose=self.verbose,
                    color=color,
                    callbacks=run_manager.get_child() if run_manager else None,
                    **tool_run_kwargs,
                )
            else:
                tool_run_kwargs = self.agent.tool_run_logging_kwargs()
                observation = await InvalidTool().arun(
                    {
                        "requested_tool_name": agent_action.tool,
                        "available_tool_names": list(name_to_tool_map.keys()),
                    },
                    verbose=self.verbose,
                    color=None,
                    callbacks=run_manager.get_child() if run_manager else None,
                    **tool_run_kwargs,
                )
            return agent_action, observation

        # Use asyncio.gather to run multiple tool.arun() calls concurrently
        result = await asyncio.gather(
            *[_aperform_agent_action(agent_action) for agent_action in actions]
        )

        return list(result)

    def _call(
        self,
        inputs: Dict[str, str],
        run_manager: Optional[CallbackManagerForChainRun] = None,
    ) -> Dict[str, Any]:
        """Run text through and get agent response."""
        print(f'inputs def call: {inputs}')
        # Construct a mapping of tool name to tool for easy lookup
        name_to_tool_map = {tool.name: tool for tool in self.tools}
        # We construct a mapping from each tool to a color, used for logging.
        color_mapping = get_color_mapping(
            [tool.name for tool in self.tools], excluded_colors=["green", "red"]
        )
        intermediate_steps: List[Tuple[AgentAction, str]] = []
        # Let's start tracking the number of iterations and time elapsed
        iterations = 0
        time_elapsed = 0.0
        start_time = time.time()
        current_instruction_index = 0
        
        # We now enter the agent loop (until it returns something).
        while self._should_continue(iterations, time_elapsed):
            next_step_output = self._take_next_step(
                name_to_tool_map,
                color_mapping,
                inputs,
                intermediate_steps,
                run_manager=run_manager,
            )
            print(f'next_step_output: {next_step_output}')
            if isinstance(next_step_output, AgentFinish):
                return self._return(
                    next_step_output, intermediate_steps, run_manager=run_manager
                )

            intermediate_steps.extend(next_step_output)
            if len(next_step_output) == 1:
                next_step_action = next_step_output[0]
                # See if tool should return directly
                tool_return = self._get_tool_return(next_step_action)
                if tool_return is not None:
                    return self._return(
                        tool_return, intermediate_steps, run_manager=run_manager
                    )
            iterations += 1
            time_elapsed = time.time() - start_time
        output = self.agent.return_stopped_response(
            self.early_stopping_method, intermediate_steps, **inputs
        )
        return self._return(output, intermediate_steps, run_manager=run_manager)

    async def _acall(
        self,
        inputs: Dict[str, str],
        run_manager: Optional[AsyncCallbackManagerForChainRun] = None,
    ) -> Dict[str, str]:
        """Run text through and get agent response."""
        print(f'inputs async def _acall: {inputs}')
        # Construct a mapping of tool name to tool for easy lookup
        name_to_tool_map = {tool.name: tool for tool in self.tools}
        # We construct a mapping from each tool to a color, used for logging.
        color_mapping = get_color_mapping(
            [tool.name for tool in self.tools], excluded_colors=["green"]
        )
        intermediate_steps: List[Tuple[AgentAction, str]] = []
        # Let's start tracking the number of iterations and time elapsed
        iterations = 0
        time_elapsed = 0.0
        start_time = time.time()
        # We now enter the agent loop (until it returns something).
        async with asyncio_timeout(self.max_execution_time):
            try:
                while self._should_continue(iterations, time_elapsed):
                    next_step_output = await self._atake_next_step(
                        name_to_tool_map,
                        color_mapping,
                        inputs,
                        intermediate_steps,
                        run_manager=run_manager,
                    )
                    if isinstance(next_step_output, AgentFinish):
                        return await self._areturn(
                            next_step_output,
                            intermediate_steps,
                            run_manager=run_manager,
                        )

                    intermediate_steps.extend(next_step_output)
                    if len(next_step_output) == 1:
                        next_step_action = next_step_output[0]
                        # See if tool should return directly
                        tool_return = self._get_tool_return(next_step_action)
                        if tool_return is not None:
                            return await self._areturn(
                                tool_return, intermediate_steps, run_manager=run_manager
                            )

                    iterations += 1
                    time_elapsed = time.time() - start_time
                output = self.agent.return_stopped_response(
                    self.early_stopping_method, intermediate_steps, **inputs
                )
                return await self._areturn(
                    output, intermediate_steps, run_manager=run_manager
                )
            except TimeoutError:
                # stop early when interrupted by the async timeout
                output = self.agent.return_stopped_response(
                    self.early_stopping_method, intermediate_steps, **inputs
                )
                return await self._areturn(
                    output, intermediate_steps, run_manager=run_manager
                )

    def _get_tool_return(
        self, next_step_output: Tuple[AgentAction, str]
    ) -> Optional[AgentFinish]:
        """Check if the tool is a returning tool."""
        agent_action, observation = next_step_output
        name_to_tool_map = {tool.name: tool for tool in self.tools}
        # Invalid tools won't be in the map, so we return False.
        if agent_action.tool in name_to_tool_map:
            if name_to_tool_map[agent_action.tool].return_direct:
                return AgentFinish(
                    {self.agent.return_values[0]: observation},
                    "",
                )
        return None

    def _prepare_intermediate_steps(
        self, intermediate_steps: List[Tuple[AgentAction, str]]
    ) -> List[Tuple[AgentAction, str]]:
        if (
            isinstance(self.trim_intermediate_steps, int)
            and self.trim_intermediate_steps > 0
        ):
            return intermediate_steps[-self.trim_intermediate_steps :]
        elif callable(self.trim_intermediate_steps):
            return self.trim_intermediate_steps(intermediate_steps)
        else:
            return intermediate_steps

## Run Agent Exexutor

In [275]:
agent_executor = AgentExecutor2.from_agent_and_tools(agent=agent, tools=tools, verbose=True)

# Running Experiments

In [276]:
article2 = webscraper.scrapeSingleUrl("https://www.straitstimes.com/asia/east-asia/japan-hopes-to-win-the-world-s-trust-over-fukushima-with-hard-science-objective-facts")
article_text_title = article2.title+article2.text

article_text_title

'Japan hopes to win the world’s trust over Fukushima with hard science, objective factsFUKUSHIMA – Japan wants to win the world’s trust that it is doing the right thing – neither wilfully poisoning the Pacific Ocean nor trying to pull the wool over people’s eyes regarding the safety of its seafood with its treated nuclear wastewater release.  On Sunday, The Straits Times was among the first media outlets – domestic and foreign – to visit the crippled Fukushima Daiichi nuclear plant since the discharge began three days earlier, in a process that will end with its full decommissioning only in 2051.  As at 5pm on Sunday (4pm Singapore time), 76 hours after the gates were opened, operator Tokyo Electric Power Co (Tepco) had released 1,420 tonnes of water that had been treated to remove radioactive materials except tritium.  Also on Sunday, Japan said tests of seawater off the coast did not detect any radioactivity. A day earlier, inspections of fish samples from waters near the plant also 

In [277]:
task = "1)From the news article, Determin Location of Disruption Event. Location should include any addresses, cities, countries, and landmarks, output an address searchable in googleMaps. Get coordinates of locaiton. 2) Identify Disruption Type of News article. 3) Identify Severity of Disruption Event."

In [278]:
agent_executor.run(task=task)



> Entering new AgentExecutor2 chain...
inputs def call: {'task': '1)From the news article, Determin Location of Disruption Event. Location should include any addresses, cities, countries, and landmarks, output an address searchable in googleMaps. Get coordinates of locaiton. 2) Identify Disruption Type of News article. 3) Identify Severity of Disruption Event.'}
def _take_next_step inputs: {'task': '1)From the news article, Determin Location of Disruption Event. Location should include any addresses, cities, countries, and landmarks, output an address searchable in googleMaps. Get coordinates of locaiton. 2) Identify Disruption Type of News article. 3) Identify Severity of Disruption Event.'}
intermediate_steps:  []
llm_output: {'Thought': 'I need to get the article first to analyze the data.', 'Action': 'get_article', 'ActionInput': '1', 'Observation': 'Analyzing...'}, type: <class 'dict'>, keys: dict_keys(['Thought', 'Action', 'ActionInput', 'Observation'])
action: get_article, act

{'Location': [35.6033123, 139.386476],
 'DisruptionType': 'Earthquake and Fire',
 'SeverityMetrics': {'HomesDestroyed': 300000, 'EconomicImpact': 'Major'}}